# Structure Composite

The Structure Composite can be used to compose a sampler that targets a particular graph structure. The ability to do this is especially useful for simulating proposed working graphs of QPU designs. In this demonstration, we will show how to build a composed sampler using the Simulated Annealing Sampler and Structure Composite, targeting a 3 dimensional simple lattice structure. 

Lattice structures are particularly interesting due to the variety of applications in the sciences. To name a few:
- Lattice-based cryptography (https://en.wikipedia.org/wiki/Lattice-based_cryptography)
- Study of crystalline structures (https://en.wikipedia.org/wiki/Crystal_structure)

First, let's use NetworkX to build a lattice-structured graph for our composed simple 3d lattice sampler, and view the lattice in a 3d plot with matplotlib.

In [ ]:
import dimod
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d

In [ ]:
# Build a simple 3D lattice with grid_graph

lattice = nx.grid_graph(dim=(4,4,4))

%matplotlib notebook

nodes_xyz = np.array(lattice.nodes)
edges_xyz = np.array(lattice.edges)

fig = plt.figure()
ax = plt.axes(projection="3d")
ax.set(xlabel="X", ylabel="Y", zlabel="Z")

#plot points
ax.scatter3D(nodes_xyz.T[0], nodes_xyz.T[1], nodes_xyz.T[2], c='b')

#plot the edges
for edge in edges_xyz:
    ax.plot3D(*zip(edge[0], edge[1]), "b-")

# Build a composed structured sampler

Now we are ready to build our composed structured sampler with our lattice graph.

Let's look at the signature of the `StructureComposite` class, found in `dimod`, to see what we need:

In [ ]:
?dimod.StructureComposite

We provide a base sampler, in this case the `SimulatedAnnealingSampler` from `neal`, a list of nodes, and a list of edges.
The edge and node lists will come from our lattice graph.

In [ ]:
from neal import SimulatedAnnealingSampler

sampler = SimulatedAnnealingSampler()

structured_sampler = dimod.StructureComposite(sampler, list(lattice.nodes), list(lattice.edges))

With the structured sampler ready, we can sample from Binary Quadratic Models with the same target graph.
Next, let's generate a random BQM that has the same graph as our structured sampler.

In [ ]:
from pprint import pprint

linear = {node:np.random.randint(-20,20) for node in lattice.nodes}
quadratic = {edge:np.random.randint(-20,20) for edge in lattice.edges}

bqm = dimod.BQM(linear, quadratic, "BINARY")

response = structured_sampler.sample(bqm, num_reads=10)

pprint(response)

# Embedding Composite

This is great! However, we are restricted to BQMs with a very specfic graph structure - one exactly matching the structured sampler. We can, however, exploit this structure in a meaningful way by composing our sampler with yet another composite - the `EmbeddingComposite`, found in the package `dwave-system`.

Looking at the signature tells us that we need to provide a *structured* sampler, among other optional arguments:

In [ ]:
from dwave.system import EmbeddingComposite

?EmbeddingComposite

Now, there may be many graphs that we can map to a lattice. Today, I though it would be interesting and perhaps motivating to show embedding the same structure of lattice, just of a smaller size. This motivates thinking of parallelization via structure.

First, let's build our composed embedding sampler, that looks to embed the input problem's graph into our original lattice graph.
Then, let's create a smaller 3d lattice to sample with the composed sampler.

In [ ]:
embedding_sampler = EmbeddingComposite(structured_sampler)

# Create a smaller simple 3d lattice
lattice_small = nx.grid_graph(dim=(2,2,2))

linear_small = {node:np.random.randint(-20,20) for node in lattice_small.nodes}
quadratic_small = {edge:np.random.randint(-20,20) for edge in lattice_small.edges}

small_bqm = dimod.BQM(linear_small, quadratic_small, "BINARY")

response_embedded = embedding_sampler.sample(small_bqm, num_reads=10, return_embedding=True)

pprint(response_embedded)

Let's visualize the embedding of the smaller lattice into the larger one:

In [ ]:
mapped = np.array(list(response_embedded.info['embedding_context']['embedding'].values()))

%matplotlib notebook

fig = plt.figure()
ax = plt.axes(projection="3d")
ax.set(xlabel="X", ylabel="Y", zlabel="Z")

#plot points
ax.scatter3D(nodes_xyz.T[0], nodes_xyz.T[1], nodes_xyz.T[2], c='b')
ax.scatter3D(mapped.T[0], mapped.T[1], mapped.T[2], s=75, c='r')

#plot the edges
for edge in edges_xyz:
    if any(np.array_equal(edge[0], x[0]) for x in mapped) and any(np.array_equal(edge[1], x[0]) for x in mapped):
        ax.plot3D(*zip(edge[0], edge[1]), "r-")
    else:
        ax.plot3D(*zip(edge[0], edge[1]), "b-")

# Composite, Composite, Composite?

To cap it off, let's take a look at another composite that acts as a post-processing layer. The `TruncateComposite` found in `dimod` allows us to filter the returned sample set, for example if we only care about a few of the lowest energy samples returned.

The signature shows us we need to provide a sampler and number of rows to return, with some other optional arguments.

In [ ]:
?dimod.TruncateComposite

In [ ]:
truncated_sampler = dimod.TruncateComposite(embedding_sampler, 5)

response_truncated = truncated_sampler.sample(small_bqm, num_reads=10)

pprint(response_truncated)